<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 2: Working with Text

Packages that are being used in this notebook:

In [1]:
from importlib.metadata import version
# 从importlib.metadata模块中导入version函数。这个函数用于获取指定库的版本信息。
print("torch version:", version("torch"))
# 调用version函数获取"torch"库的版本，并将其打印出来。这里会输出类似 "torch version: x.x.x" 的信息，
print("tiktoken version:", version("tiktoken"))
# 同样调用version函数获取"tiktoken"库的版本，并将其打印出来。

PackageNotFoundError: No package metadata was found for torch

- This chapter covers data preparation and sampling to get input data "ready" for the LLM

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 Understanding word embeddings

- No code in this section

- There are many forms of embeddings; we focus on text embeddings in this book

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLMs work with embeddings in high-dimensional spaces (i.e., thousands of dimensions)
- Since we can't visualize such high-dimensional spaces (we humans think in 1, 2, or 3 dimensions), the figure below illustrates a 2-dimensional embedding space

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 Tokenizing text

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [2]:
import os
import urllib.request
# 检查当前目录下是否不存在名为"the-verdict.txt"的文件
if not os.path.exists("the-verdict.txt"):
    # 定义要下载文件的URL地址
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    # 定义要保存下载文件的本地路径，这里是当前目录下的"the-verdict.txt"
    file_path = "the-verdict.txt"
    # 使用urllib.request库的urlretrieve函数从指定的URL下载文件，并保存到本地指定的路径
    urllib.request.urlretrieve(url, file_path)

- (If you encounter an `ssl.SSLCertVerificationError` when executing the previous code cell, it might be due to using an outdated Python version; you can find [more information here on GitHub](https://github.com/rasbt/LLMs-from-scratch/pull/403))

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    # 使用`with`语句打开名为"the-verdict.txt"的文件，以只读模式（"r"）打开，并指定编码为"utf-8"。
    # `with`语句会自动处理文件的打开和关闭操作，确保在代码块执行完毕后正确关闭文件，避免资源泄露。
    # 这里将打开的文件对象赋值给变量`f`，以便后续操作使用。
    raw_text = f.read()
# 打印出读取到的文本的总字符数。通过调用`len()`函数获取`raw_text`的长度，也就是字符的数量。
print("Total number of character:", len(raw_text))
# 打印出读取到的文本的前99个字符。通过切片操作`raw_text[:99]`获取从文本开头到第99个字符（不包括第99个字符）的内容。
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- The goal is to tokenize and embed this text for an LLM
- Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above
- The following regular expression will split on whitespaces

In [4]:
import re
# 定义一个字符串变量text，其中包含了一些文本内容，这里是"Hello, world. This, is a test."
text = "Hello, world. This, is a test."
# 使用re模块的split函数对text字符串进行分割操作。
# 这里的正则表达式r'(\s)'表示以空白字符（包括空格、制表符、换行符等）作为分割点。
# 并且由于使用了括号将\s括起来，所以分割时会保留分割点的字符，也就是空白字符会被保留在分割后的结果列表中。
result = re.split(r'(\s)', text)
# 打印出分割后的结果列表。这个列表中包含了按照空白字符分割后的各个子字符串以及被保留的空白字符。
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- We don't only want to split on whitespaces but also commas and periods, so let's modify the regular expression to do that as well

In [5]:
import re
text = "Hello, world. This, is a test."
# 使用re模块的split函数对text字符串进行分割操作。
# 这里的正则表达式r'([,.]|\s)'含义如下：
# - '|' 是正则表达式中的或运算符，表示匹配其中任意一种情况。
# - '([,.])' 表示匹配逗号或句号，并且由于使用了括号，会将匹配到的逗号或句号保留在分割后的结果中。
# - '\s' 表示匹配空白字符（包括空格、制表符、换行符等），同样也会将匹配到的空白字符保留在分割后的结果中。
# 所以整体上这个正则表达式会以逗号、句号或者空白字符作为分割点对text字符串进行分割，并且保留这些分割点的字符。
result = re.split(r'([,.]|\s)', text)
# 打印出分割后的结果列表。这个列表中将包含按照逗号、句号或者空白字符分割后的各个子字符串以及被保留的逗号、句号和空白字符。
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- As we can see, this creates empty strings, let's remove them

In [6]:
# Strip whitespace from each item and then filter out any empty strings.
# 对于result列表中的每一个元素item，先使用strip()方法去除其两端的空白字符（包括空格、制表符、换行符等），
# 然后通过if item.strip()条件进行过滤，只有当去除空白字符后item不为空字符串时，才会将其保留在新的列表中。
result = [item for item in result if item.strip()]
# 打印经过上述处理后的result列表，此时列表中的元素都已经去除了两端空白字符并且不存在空字符串了。
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- This looks pretty good, but let's also handle other types of punctuation, such as periods, question marks, and so on

In [7]:
text = "Hello, world. Is this-- a test?"
# - '([,.:;?_!"()\'])' 表示匹配逗号、句号、冒号、分号、问号、感叹号、双引号、单引号、左括号、右括号这些标点符号，
#   并且由于使用了括号，会将匹配到的标点符号保留在分割后的结果中。
# - '--' 表示匹配连续的两个减号，同样会将其保留在分割后的结果中。
# - '\s' 表示匹配空白字符（包括空格、制表符、换行符等），也会将匹配到的空白字符保留在分割后的结果中。
# 所以整体上这个正则表达式会以列举的这些标点符号、连续两个减号或者空白字符作为分割点对text字符串进行分割，
# 并且保留这些分割点的字符。
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
# 以下这行代码使用了列表推导式来对刚才得到的result列表进行进一步处理。
# 对于result列表中的每一个元素item，先使用strip()方法去除其两端的空白字符（包括空格、制表符、换行符等），
# 然后通过if item.strip()条件进行过滤，只有当去除空白字符后item不为空字符串时，才会将其保留在新的列表中。
result = [item.strip() for item in result if item.strip()]
# 打印经过上述处理后的result列表，此时列表中的元素都已经去除了两端空白字符并且不存在空字符串了，
# 并且这些元素是按照之前设定的标点符号、连续两个减号或空白字符分割原始文本得到的结果。
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- This is pretty good, and we are now ready to apply this tokenization to the raw text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
# 使用re模块的split函数对raw_text字符串进行分割操作。
# 正则表达式r'([,.:;?_!"()\']|--|\s)'的含义如下：
# - '|' 是正则表达式中的“或”运算符，表示匹配其中任意一种情况。
# - '([,.:;?_!"()\'])' 表示匹配逗号、句号、冒号、分号、问号、感叹号、双引号、单引号、左括号、右括号这些标点符号，
#   并且由于使用了括号，会将匹配到的标点符号保留在分割后的结果中。
# - '--' 表示匹配连续的两个减号，同样会将其保留在分割后的结果中。
# - '\s' 表示匹配空白字符（包括空格、制表符、换行符等），也会将匹配到的空白字符保留在分割后的结果中。
# 所以整体上这个正则表达式会以列举的这些标点符号、连续两个减号或者空白字符作为分割点对raw_text字符串进行分割，
# 并且保留这些分割点的字符。
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# 以下这行代码使用了列表推导式来对刚才得到的preprocessed列表进行进一步处理。
# 对于preprocessed列表中的每一个元素item，先使用strip()方法去除其两端的空白字符（包括空格、制表符、换行符等），
# 然后通过if item.strip()条件进行过滤，只有当去除空白字符后item不为空字符串时，才会将其保留在新的列表中。
preprocessed = [item.strip() for item in preprocessed if item.strip()]
# 打印经过上述处理后的preprocessed列表的前30个元素。
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- Let's calculate the total number of tokens

In [9]:
print(len(preprocessed))

4690


## 2.3 Converting tokens into token IDs

- Next, we convert the text tokens into token IDs that we can process via embedding layers later

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- From these tokens, we can now build a vocabulary that consists of all the unique tokens

In [10]:
# set()函数会将preprocessed列表中的元素转换为集合，集合的特点是元素具有唯一性，即会自动去除重复的元素。
# 所以这里通过set(preprocessed)就得到了一个包含preprocessed中所有不重复元素的集合。
# 然后再使用sorted()函数对这个集合进行排序，排序后的结果是一个按照某种顺序（比如字母顺序等）排列的列表，
# 其中包含了preprocessed中所有不重复的元素，将其赋值给all_words变量。
all_words = sorted(set(preprocessed))
# 计算词汇表的大小，也就是all_words这个列表的长度。
# 由于all_words列表包含了经过处理后的文本中所有不重复的单词（或其他文本单元），
# 所以其长度就代表了词汇表中不同单词（或文本单元）的数量，将其赋值给vocab_size变量。
vocab_size = len(all_words)
# 打印出词汇表的大小，即显示出经过前面的处理后，文本中不同单词（或文本单元）的数量。
print(vocab_size)

1130


In [11]:
# 使用字典推导式来创建一个名为vocab的字典。
# 在这个字典推导式中：
# - enumerate(all_words) 会对all_words列表进行枚举操作。
#   枚举操作会返回一个包含索引（从0开始）和对应元素的元组序列。
#   例如，如果all_words = ['apple', 'banana', 'cherry']，那么enumerate(all_words)会返回[(0, 'apple'), (1, 'banana'), (2, 'cherry')]。
# - token:integer 表示字典的键值对形式，这里将枚举得到的元组中的元素（即all_words中的单词，作为token）作为键，
#   而将枚举得到的元组中的索引（作为integer）作为值。
# 所以整体上这行代码创建了一个字典vocab，其中键是all_words列表中的各个单词（或其他文本单元），
# 值是这些单词（或其他文本单元）在all_words列表中的索引位置。
vocab = {token:integer for integer,token in enumerate(all_words)}

- Below are the first 50 entries in this vocabulary:

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- Below, we illustrate the tokenization of a short sample text using a small vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- Putting it now all together into a tokenizer class

In [13]:
class SimpleTokenizerV1:
    # 定义一个名为SimpleTokenizerV1的类，用于实现简单的文本编码和解码功能。
    def __init__(self, vocab):
        # 类的初始化方法，接收一个词汇表（vocab）作为参数。
        self.str_to_int = vocab
        # 将传入的词汇表（vocab）赋值给实例变量str_to_int，这个字典用于将字符串（单词等文本单元）映射为整数索引。
        self.int_to_str = {i:s for s,i in vocab.items()}
        # 通过字典推导式创建一个新的字典int_to_str，它是将词汇表（vocab）中的键值对进行反转得到的。
        # 即原来词汇表中以字符串为键、整数为值，现在这个字典以整数为键、字符串为值，用于将整数索引映射回字符串。
    def encode(self, text):
        # 定义一个名为encode的方法，用于对输入的文本进行编码操作。
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # 使用re模块的split函数对输入的文本（text）进行分割操作。
        # 正则表达式r'([,.:;?_!"()\']|--|\s)'会以逗号、句号、冒号、分号、问号、感叹号、双引号、单引号、左括号、右括号、连续两个减号或者空白字符作为分割点对文本进行分割，
        # 并且保留这些分割点的字符。
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # 使用列表推导式对分割后的结果进行处理。
        # 对于分割后的每个元素（item），先使用strip()方法去除其两端的空白字符（包括空格、制表符、换行符等），
        # 然后通过if item.strip()条件进行过滤，只有当去除空白字符后item不为空字符串时，才会将其保留在新的列表中。
        ids = [self.str_to_int[s] for s in preprocessed]
        # 通过列表推导式将经过处理后的每个文本单元（preprocessed中的元素），根据str_to_int字典将其映射为对应的整数索引，
        # 并将这些整数索引组成一个列表（ids）。
        return ids
        # 返回编码后的整数索引列表（ids）。
    def decode(self, ids):
        # 定义一个名为decode的方法，用于对输入的整数索引列表进行解码操作。
        text = " ".join([self.int_to_str[i] for i in ids])
        # 通过列表推导式将输入的整数索引列表（ids）中的每个整数（i），根据int_to_str字典将其映射回对应的字符串，
        # 然后使用" ".join()方法将这些字符串连接起来，中间用空格分隔，得到一个初步解码后的文本（text）。
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        # 使用re模块的sub函数对初步解码后的文本（text）进行替换操作。
        # 正则表达式r'\s+([,.?!"()\'])'会匹配一个或多个空白字符（\s+）后面跟着逗号、句号、问号、感叹号、双引号、单引号、左括号、右括号这些标点符号（[,.?!"()\']）的情况。
        # 然后将匹配到的内容替换为括号内捕获的标点符号（r'\1'，这里的\1表示引用前面正则表达式中捕获组所捕获的内容，即标点符号），
        # 目的是去除标点符号前面多余的空格。

        return text
        # 返回经过处理后的解码文本（text）。

- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [14]:
tokenizer = SimpleTokenizerV1(vocab)
# 创建一个SimpleTokenizerV1类的实例tokenizer，传入之前定义好的词汇表vocab作为参数。
# 这样就初始化好了一个可以对文本进行编码和解码操作的tokenizer对象。
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
# 调用tokenizer对象的encode方法对text文本进行编码操作。
# 这个方法会按照SimpleTokenizerV1类中定义的编码逻辑，先对文本进行预处理（如分割、清理等），
# 然后根据词汇表vocab将文本中的每个单词或文本单元映射为对应的整数索引，最后返回这些整数索引组成的列表ids。
print(ids)
# 打印出经过编码后得到的整数索引列表ids，通过查看这个列表可以了解文本在经过特定编码规则处理后，
# 每个文本单元所对应的整数表示形式。

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- We can decode the integers back into text

In [15]:
tokenizer.decode(ids)
#解码

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))
#先编码后解码

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Adding special context tokens

- It's useful to add some "special" tokens for unknown words and to denote the end of a text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- Some tokenizers use special tokens to help the LLM with additional context
- Some of these special tokens are
  - `[BOS]` (beginning of sequence) marks the beginning of text
  - `[EOS]` (end of sequence) marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on)
  - `[PAD]` (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length)
- `[UNK]` to represent words that are not included in the vocabulary

- Note that GPT-2 does not need any of these tokens mentioned above but only uses an `<|endoftext|>` token to reduce complexity
- The `<|endoftext|>` is analogous to the `[EOS]` token mentioned above
- GPT also uses the `<|endoftext|>` for padding (since we typically use a mask when training on batched inputs, we would not attend padded tokens anyways, so it does not matter what these tokens are)
- GPT-2 does not use an `<UNK>` token for out-of-vocabulary words; instead, GPT-2 uses a byte-pair encoding (BPE) tokenizer, which breaks down words into subword units which we will discuss in a later section



- We use the `<|endoftext|>` tokens between two independent sources of text:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- Let's see what happens if we tokenize the following text:

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- The above produces an error because the word "Hello" is not contained in the vocabulary
- To deal with such cases, we can add special tokens like `"<|unk|>"` to the vocabulary to represent unknown words
- Since we are already extending the vocabulary, let's add another token called `"<|endoftext|>"` which is used in GPT-2 training to denote the end of a text (and it's also used between concatenated text, like if our training datasets consists of multiple articles, books, etc.)

In [18]:
all_tokens = sorted(list(set(preprocessed)))
# 向all_tokens列表中添加两个特殊的标记："<|endoftext|>"和"<|unk|>"。
# "<|endoftext|>"通常用于表示文本的结束位置，在很多文本处理任务中，比如生成文本时，用于标识一段文本生成完毕。
# "<|unk|>"通常用于表示未知的单词或文本单元，当在处理文本过程中遇到不在词汇表中的单词时，就可以用"<|unk|>"来标记。
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
# - enumerate(all_tokens)会对all_tokens列表进行枚举操作。
#   枚举操作会返回一个包含索引（从0开始）和对应元素的元组序列。
#   例如，如果all_tokens = ['apple', 'banana', 'cherry']，那么enumerate(all_tokens)会返回[(0, 'apple'), (1, 'banana'), (2, 'cherry')]。
# - token:integer表示字典的键值对形式，这里将枚举得到的元组中的元素（即all_tokens中的单词或标记，作为token）作为键，
#   而将枚举得到的元组中的索引（作为integer）作为值。
# 所以整体上这行代码创建了一个字典vocab，其中键是all_tokens列表中的各个单词（或标记），
# 值是这些单词（或标记）在all_tokens列表中的索引位置。
vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- We also need to adjust the tokenizer accordingly so that it knows when and how to use the new `<unk>` token

In [21]:
class SimpleTokenizerV2:
    # 定义一个名为SimpleTokenizerV2的类，用于实现文本的编码和解码功能。
    def __init__(self, vocab):
        # 类的初始化方法，接收一个词汇表（vocab）作为参数。
        self.str_to_int = vocab
        # 将传入的词汇表（vocab）赋值给实例变量str_to_int，这个字典用于将字符串（单词等文本单元）映射为整数索引。
        # 方便在编码过程中将文本单元转换为对应的整数表示。
        self.int_to_str = { i:s for s,i in vocab.items()}
        # 通过字典推导式创建一个新的字典int_to_str，它是将词汇表（vocab）中的键值对进行反转得到的。
        # 即原来词汇表中以字符串为键、整数为值，现在这个字典以整数为键、字符串为值。
        # 用于在解码过程中将整数索引转换回对应的字符串表示。
    def encode(self, text):
        # 定义一个名为encode的方法，用于对输入的文本进行编码操作，将文本转换为对应的整数索引列表。
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # 使用re模块的split函数对输入的文本（text）进行分割操作。
        # 正则表达式r'([,.:;?_!"()\']|--|\s)'会以逗号、句号、冒号、分号、问号、感叹号、双引号、单引号、左括号、右括号、
        # 连续两个减号或者空白字符作为分割点对文本进行分割，并且保留这些分割点的字符。
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # 使用列表推导式对分割后的结果进行处理。
        # 对于分割后的每个元素（item），先使用strip()方法去除其两端的空白字符（包括空格、制表符、换行符等），
        # 然后通过if item.strip()条件进行过滤，只有当去除空白字符后item不为空字符串时，才会将其保留在新的列表中。
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        # 再次使用列表推导式对经过上一步处理后的preprocessed列表进行处理。
        # 对于列表中的每个元素item，如果item在self.str_to_int字典中有对应的映射（即词汇表中存在该单词或文本单元），
        # 则保持item不变；如果item不在词汇表中，就将其替换为特殊标记"<|unk|>"，用于表示未知的单词或文本单元。
        ids = [self.str_to_int[s] for s in preprocessed]
        # 通过列表推导式将经过上述处理后的每个文本单元（preprocessed中的元素），根据str_to_int字典将其映射为对应的整数索引，
        # 并将这些整数索引组成一个列表（ids）。
        return ids
        # 返回编码后的整数索引列表（ids）。
    def decode(self, ids):
        # 定义一个名为decode的方法，用于对输入的整数索引列表进行解码操作，将整数索引列表转换回原始文本形式。
        text = " ".join([self.int_to_str[i] for i in ids])
        # 通过列表推导式将输入的整数索引列表（ids）中的每个整数（i），根据int_to_str字典将其映射回对应的字符串，
        # 然后使用" ".join()方法将这些字符串连接起来，中间用空格分隔，得到一个初步解码后的文本（text）。
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        # 使用re模块的sub函数对初步解码后的文本（text）进行替换操作。
        # 正则表达式r'\s+([,.:;?!"()\'])'会匹配一个或多个空白字符（\s+）后面跟着逗号、句号、问号、感叹号、双引号、单引号、左括号、右括号这些标点符号（[,.:;?!"()\'])的情况。
        # 然后将匹配到的内容替换为括号内捕获的标点符号（r'\1'，这里的\1表示引用前面正则表达式中捕获组所捕获的内容，即标点符号），
        # 目的是去除标点符号前面多余的空格。
        return text
        # 返回经过处理后的解码文本（text）。

Let's try to tokenize text with the modified tokenizer:

In [22]:
tokenizer = SimpleTokenizerV2(vocab)
# 创建一个SimpleTokenizerV2类的实例tokenizer，传入之前定义好的词汇表vocab作为参数。
# 这样就初始化好了一个可以对文本进行编码和解码操作的tokenizer对象，它将按照SimpleTokenizerV2类中的逻辑来处理文本。
text1 = "Hello, do you like tea?"
# 定义一个字符串变量text1，包含了一段询问是否喜欢茶的简单文本内容。
text2 = "In the sunlit terraces of the palace."
# 定义另一个字符串变量text2，描述了在宫殿阳光照耀的露台的场景的文本内容。
text = " <|endoftext|> ".join((text1, text2))
# 使用字符串的join方法将text1和text2这两个字符串连接起来，中间用" <|endoftext|> "作为分隔符。
# 这里的" <|endoftext|> "是一个特殊的标记，通常用于表示一段文本的结束位置，在这个例子中用于分隔两段不同的文本内容。
print(text)
# 打印出连接后的文本text，通过查看这个输出可以看到两段文本按照设定的分隔符连接在一起的结果。

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- In this chapter, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
- I created a notebook in the [./bytepair_encoder](../02_bonus_bytepair-encoder) that compares these two implementations side-by-side (tiktoken was about 5x faster on the sample text)

In [25]:
# pip install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")
# 调用tiktoken库中的get_encoding函数，并传入参数"gpt2"。
# tiktoken是一个用于处理文本分词和编码的库，在OpenAI相关的应用以及很多自然语言处理任务中较为常用。
# get_encoding函数的作用是根据传入的编码名称（这里是"gpt2"）来获取对应的文本编码方式的tokenizer对象。
# 这个tokenizer对象可以用于将文本按照特定的规则进行分词处理，并将每个词（token）转换为对应的编码表示形式，以便后续在模型中进行处理，
# 例如输入到基于GPT架构的语言模型等相关任务中。
# 将获取到的tokenizer对象赋值给变量tokenizer，以便后续使用该tokenizer对文本进行操作。

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
# 调用之前获取到的tokenizer对象（假设已经通过如tiktoken.get_encoding("gpt2")等方式获取）的encode方法对text文本进行编码操作。
# encode方法会将输入的文本按照tokenizer所基于的编码规则（这里是基于"gpt2"编码方式相关的规则）进行分词处理，
# 并将每个分词（token）转换为对应的整数编码表示形式。
# 这里传入了allowed_special={"<|endoftext|>"}参数，它表示允许在编码过程中把特殊标记 "<|endoftext|>" 当作特殊情况进行处理，
# 通常是确保这个特殊标记能被正确编码而不被当作普通文本处理。
print(integers)
# 打印出经过编码后得到的整数列表integers，通过查看这个列表可以了解文本在经过特定编码规则处理后，
# 每个分词（token）所对应的整数表示形式，这些整数编码可以用于后续在相关的自然语言处理模型或任务中进行输入等操作。

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE tokenizers break down unknown words into subwords and individual characters:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 Data sampling with a sliding window

- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- For each text chunk, we want the inputs and targets
- Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

In [31]:
enc_sample = enc_text[50:]
# 从enc_text这个序列（可能是列表、字符串、数组等可切片的数据结构，具体要看enc_text的定义）中进行切片操作。
# 这里的切片操作 [50:] 表示选取从索引为50的元素开始一直到序列末尾的所有元素。
# 将切片得到的部分赋值给新的变量enc_sample，以便后续对这部分数据进行单独的处理或分析等操作。

In [32]:
context_size = 4
# 定义一个变量context_size，并赋值为4。
x = enc_sample[:context_size]
# 从enc_sample序列中选取一部分作为x。
# 这里的切片操作[:context_size]表示选取从enc_sample序列开头到索引为context_size - 1（因为切片是左闭右开区间）的元素，
# 也就是选取enc_sample序列的前context_size个元素作为x。
y = enc_sample[1:context_size + 1]
# 同样从enc_sample序列中选取一部分作为y。
# 这里的切片操作[1:context_size + 1]表示选取从enc_sample序列索引为1的元素开始，到索引为context_size（因为是左闭右开区间，所以实际选取到索引为context_size的元素）的元素，
# 也就是选取enc_sample序列中从第二个元素开始到第context_size + 1个元素作为y。
print(f"x: {x}")
# 使用f-string格式化字符串的方式打印出变量x的值，在输出中会显示"x: "以及x的值，方便查看x所代表的序列片段内容。
print(f"y:      {y}")
# 同样使用f-string格式化字符串的方式打印出变量y的值，在输出中会显示"y: "以及y的值，并且前面有一些空格用于格式上的区分，方便查看y所代表的序列片段内容。

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- One by one, the prediction would look like as follows:

In [33]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    # 这里的切片操作[:i]表示选取从enc_sample序列开头到索引为i - 1（因为切片是左闭右开区间）的元素，
    # 也就是随着循环i的变化，会依次选取enc_sample序列的前1个、前2个、前3个……直到前context_size个元素作为不同长度的上下文。
    desired = enc_sample[i]
    # 从enc_sample序列中选取索引为i的元素作为期望输出（desired）。
    # 结合上面选取上下文的操作，这里可以理解为对于每一个选定的上下文长度，选取该上下文之后紧接着的那个元素作为期望输出，
    print(context, "---->", desired)
    # 打印出当前选定的上下文（context）以及对应的期望输出（desired），中间用"---->"进行分隔，方便查看和对比不同上下文长度下的情况。

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- We will take care of the next-word prediction in a later chapter after we covered the attention mechanism
- For now, we implement a simple data loader that iterates over the input dataset and returns the inputs and targets shifted by one

- Install and import PyTorch (see Appendix A for installation tips)

In [4]:
import torch
print("PyTorch version:", torch.__version__)

ModuleNotFoundError: No module named 'torch'

- We use a sliding window approach, changing the position by +1:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- Create dataset and dataloader that extract chunks from the input text dataset

In [36]:
from torch.utils.data import Dataset, DataLoader
# 从torch.utils.data模块中导入Dataset和DataLoader类。
# Dataset是PyTorch中用于表示数据集的抽象基类，继承它并实现特定方法可定义自己的数据集。
# DataLoader用于加载数据集，能方便地对数据集进行批量加载、打乱顺序等操作，便于在模型训练等过程中使用。
class GPTDatasetV1(Dataset):
    # 定义名为GPTDatasetV1的类，继承自Dataset类，用于创建适合GPT（Generative Pretrained Transformer）模型训练的数据集。
    def __init__(self, txt, tokenizer, max_length, stride):
        # 类的初始化方法，接收以下参数：
        # - txt：要处理的原始文本内容。
        # - tokenizer：用于对文本进行编码的分词器对象，可将文本转换为对应的编码表示形式（如将单词转换为整数编码等）。
        # - max_length：每个数据样本的最大长度，即把原始文本划分成固定长度片段时的长度限制。
        # - stride：滑动窗口的步长，用于在对原始文本进行划分时控制相邻片段之间的重叠程度。
        self.input_ids = []
        # 初始化一个空列表self.input_ids，用于存储划分后的输入数据的编码表示形式，这些编码将作为模型的输入。
        self.target_ids = []
        # 初始化一个空列表self.target_ids，用于存储划分后的目标数据的编码表示形式，这些编码将作为模型训练时要预测的目标。
        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        # 使用传入的tokenizer对象对整个原始文本txt进行编码操作。
        # 通过调用tokenizer的encode方法，将原始文本按tokenizer的编码规则转换为对应的编码序列（如整数编码序列）。
        # 传入allowed_special={"<|endoftext|>"}参数，确保特殊标记 "<|endoftext|>" 能被正确编码处理，不被当作普通文本对待。
        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            # 开启一个循环，通过滑动窗口方式对编码后的文本序列token_ids进行划分操作。
            # 循环变量i从0开始，每次增加stride步长，直到满足len(token_ids) - max_length条件（确保能划分出完整长度为max_length的片段）。
            input_chunk = token_ids[i:i + max_length]
            # 从编码后的文本序列token_ids中选取一段作为输入数据的编码表示形式。
            # 通过切片操作i:i + max_length选取从索引为i到索引为i + max_length - 1的元素，得到长度为max_length的片段，作为输入数据。
            target_chunk = token_ids[i + 1: i + max_length + 1]
            # 从编码后的文本序列token_ids中选取一段作为目标数据的编码表示形式。
            # 通过切片操作i + 1: i + max_length + 1选取从索引为i + 1到索引为i + max_length的元素，得到长度为max_length的片段，作为目标数据。
            # 注意目标数据的选取是相对于输入数据向后偏移了1个元素，因在训练模型时，通常基于当前输入预测下一个元素。
            self.input_ids.append(torch.tensor(input_chunk))
            # 将输入数据的编码表示形式转换为PyTorch的张量（tensor）形式，并添加到self.input_ids列表中。
            # 这样self.input_ids列表中就存储了一系列划分好的输入数据的张量表示。
            self.target_ids.append(torch.tensor(target_chunk))
            # 将目标数据的编码表示形式转换为PyTorch的张量（tensor）形式，并添加到self.target_ids列表中。
            # 这样self.target_ids列表中就存储了一系列划分好的目标数据的张量表示。
    def __len__(self):
        return len(self.input_ids)
        # 重写Dataset类的__len__方法，用于返回数据集的长度，即self.input_ids列表的长度。
        # 因self.input_ids列表存储了所有划分好的输入数据的张量表示，其长度代表数据集的样本数量。
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
        # 重写Dataset类的__getitem__方法，用于根据给定的索引idx获取数据集中的一个样本。
        # 返回一个元组，包含对应索引的输入数据的张量表示（self.input_ids[idx]）和目标数据的张量表示（self.target_ids[idx]）。
        # 这样在使用DataLoader加载数据集时，可通过索引方便地获取每个样本的数据。

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    # 定义一个名为create_dataloader_v1的函数，用于创建适合GPT模型训练的数据加载器（DataLoader）。
    # 该函数接收以下参数：
    # - txt：要处理的原始文本内容，将基于此文本创建数据集并加载数据。
    # - batch_size：每个批次的大小，即每次从数据集中取出多少个样本进行处理，默认为4。
    # - max_length：每个数据样本的最大长度，用于将原始文本划分成固定长度片段，默认为256。
    # - stride：滑动窗口的步长，控制在划分原始文本时相邻片段之间的重叠程度，默认为128。
    # - shuffle：是否打乱数据集的顺序，默认为True，打乱顺序有助于提高模型训练的效果。
    # - drop_last：是否丢弃最后一个不完整的批次，默认为True，如果最后一个批次的样本数量不足batch_size，则丢弃。
    # - num_workers：用于数据加载的工作线程数量，默认为0，表示在主线程中加载数据。
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")
    # 初始化分词器（tokenizer）。
    # 通过调用tiktoken库的get_encoding函数，并传入参数"gpt2"，获取基于"gpt2"编码方式的分词器对象。
    # 这个分词器将用于对原始文本进行编码，把文本转换为适合模型处理的编码表示形式。
    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    # 创建数据集对象。
    # 调用GPTDatasetV1类的构造函数，传入原始文本txt、刚初始化的分词器tokenizer、设定的每个数据样本的最大长度max_length以及滑动窗口的步长stride。
    # GPTDatasetV1类会对原始文本进行编码、划分等操作，将其转化为适合GPT模型训练的数据集格式。
    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    # 创建数据加载器（DataLoader）对象。
    # 将前面创建的数据集对象dataset传入DataLoader的构造函数，同时设置以下参数：
    # - batch_size：指定每个批次的大小，与传入函数的batch_size参数值相同。
    # - shuffle：是否打乱数据集顺序，与传入函数的shuffle参数值相同。
    # - drop_last：是否丢弃最后一个不完整的批次，与传入函数的drop_last参数值相同。
    # - num_workers：用于数据加载的工作线程数量，与传入函数的num_workers参数值相同。
    # 数据加载器会根据这些设置，方便地对数据集进行批量加载、打乱顺序（如果设置为True）等操作，以便于在GPT模型训练等过程中使用。
    return dataloader
    # 返回创建好的数据加载器对象dataloader，以便在其他地方（如模型训练代码中）使用该数据加载器来获取数据进行训练等操作。

- Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)
# 调用create_dataloader_v1函数来创建一个数据加载器（dataloader）对象。
# 传入原始文本raw_text作为要处理的数据内容。
# 同时设置了以下参数：
# - batch_size=1：表示每个批次只包含1个样本，这样每次从数据集中获取数据时就是逐个样本进行的。
# - max_length=4：每个数据样本的最大长度设置为4，即数据集中划分出的每个片段长度最多为4个编码单位（例如可能是4个单词经过编码后的形式）。
# - stride=1：滑动窗口的步长设置为1，意味着在对原始文本进行划分成片段时，相邻片段之间的重叠部分只有1个编码单位。
# - shuffle=False：不打乱数据集中样本的顺序，按照原始文本在经过划分等操作后的顺序来获取数据。
data_iter = iter(dataloader)
# 通过调用iter函数将创建好的数据加载器dataloader转换为一个迭代器（data_iter）。
# 这样就可以使用迭代器的方式来逐个获取数据加载器中的批次数据。
first_batch = next(data_iter)
# 使用next函数从迭代器data_iter中获取下一个批次的数据，由于前面设置batch_size=1，所以这里获取到的就是第一个且唯一的一个样本数据。
# 将获取到的第一个批次的数据赋值给first_batch变量，以便后续对其进行查看、分析等操作。
print(first_batch)
# 打印出获取到的第一个批次的数据first_batch，通过查看输出可以了解数据在经过前面一系列处理（如文本编码、划分、组成批次等）后呈现出的具体形式，
# 通常会是一个包含输入数据编码和目标数据编码的元组形式（具体取决于create_dataloader_v1函数内部创建数据集时的设置）。

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
# 再次使用next函数从已经创建好的迭代器data_iter中获取下一个批次的数据。
print(second_batch)
# 打印出获取到的第二个批次的数据second_batch。
# 通过查看输出可以了解数据在经过前面一系列处理（如文本编码、划分、组成批次等）后呈现出的具体形式，

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- An example using stride equal to the context length (here: 4) as shown below:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- We can also create batched outputs
- Note that we increase the stride here so that we don't have overlaps between the batches, since more overlap could lead to increased overfitting

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
# 调用create_dataloader_v1函数来创建一个数据加载器（dataloader）对象。
# 传入原始文本raw_text作为要处理的数据内容。
# 同时设置了以下参数：
# - batch_size = 8：表示每个批次将包含8个样本，这样每次从数据集中获取数据时会一次性获取8个经过处理的样本数据。
# - max_length = 4：每个数据样本的最大长度设置为4，即数据集中划分出的每个片段长度最多为4个编码单位（例如可能是4个单词经过编码后的形式）。
# - stride = 4：滑动窗口的步长设置为4，意味着在对原始文本进行划分成片段时，相邻片段之间的重叠部分为4个编码单位，这里步长与最大长度相同，可能会导致划分出的片段之间没有重叠或者有特殊的重叠情况，具体取决于原始文本的长度和编码方式等因素。
# - shuffle = False：不打乱数据集中样本的顺序，按照原始文本在经过划分等操作后的顺序来获取数据。
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
# 使用next函数从迭代器data_iter中获取下一个批次的数据。
# 由于前面设置batch_size = 8，所以这里获取到的就是一个包含8个样本数据的批次，并且将这个批次中的输入数据部分赋值给inputs变量，目标数据部分赋值给targets变量。
# 具体来说，inputs和targets的格式和内容取决于create_dataloader_v1函数内部创建数据集时的设置，通常inputs是用于模型输入的编码数据，targets是模型要预测的目标编码数据。
print("Inputs:\n", inputs)
# 打印出获取到的批次数据中的输入数据部分inputs。
# 在输出中会先显示"Inputs:\n"作为提示，然后显示inputs的具体内容，以便查看输入数据在经过前面一系列处理（如文本编码、划分、组成批次等）后呈现出的具体形式。
print("\nTargets:\n", targets)
# 打印出获取到的批次数据中的目标数据部分targets。
# 在输出中会先显示"\nTargets:\n"作为提示，然后显示targets的具体内容，以便查看目标数据在经过前面一系列处理（如文本编码、划分、组成批次等）后呈现出的具体形式。

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating token embeddings

- The data is already almost ready for an LLM
- But lastly let us embed the tokens in a continuous vector representation using an embedding layer
- Usually, these embedding layers are part of the LLM itself and are updated (trained) during model training

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- Suppose we have the following four input examples with input ids 2, 3, 5, and 1 (after tokenization):

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])

- For the sake of simplicity, suppose we have a small vocabulary of only 6 words and we want to create embeddings of size 3:

In [43]:
vocab_size = 6
# 定义一个变量vocab_size，并赋值为6，表示词汇表的大小为6，即总共包含6个不同的词汇（或其他可编码的文本单元等）。
output_dim = 3
# 定义一个变量output_dim，并赋值为3，表示输出的维度为3，这个输出维度通常与后续要进行的某种操作（比如经过神经网络层后的输出维度等）相关。
torch.manual_seed(123)
# 使用torch库的manual_seed函数设置随机种子为123。
# 设置随机种子的目的是为了在进行涉及随机操作（如神经网络的初始化等可能涉及随机因素的情况）时，能够复现相同的随机结果。
# 也就是说，只要使用相同的随机种子，每次运行代码时在涉及随机的部分都会产生相同的结果，方便调试和对比不同运行情况。
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# 创建一个嵌入层（Embedding Layer）对象embedding_layer。
# 这里使用torch.nn模块中的Embedding类来创建嵌入层。
# 传入的参数vocab_size表示词汇表的大小，即要嵌入的词汇数量；output_dim表示嵌入后的输出维度，即每个词汇经过嵌入层处理后将被映射到一个output_dim维的向量空间中。
# 嵌入层的作用通常是将离散的词汇（或其他可编码的文本单元等）转换为连续的向量表示，以便在后续的神经网络处理中更好地进行运算和分析等操作。

- This would result in a 6x3 weight matrix:

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- For those who are familiar with one-hot encoding, the embedding layer approach above is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully-connected layer, which is described in the supplementary code in [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)
- Because the embedding layer is just a more efficient implementation that is equivalent to the one-hot encoding and matrix-multiplication approach it can be seen as a neural network layer that can be optimized via backpropagation

- To convert a token with id 3 into a 3-dimensional vector, we do the following:

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- Note that the above is the 4th row in the `embedding_layer` weight matrix
- To embed all four `input_ids` values above, we do

In [46]:
print(embedding_layer(input_ids))
# 调用之前创建的嵌入层对象embedding_layer，并将input_ids作为参数传入。
# input_ids应该是一个包含词汇索引的张量
# 嵌入层embedding_layer会根据传入的这些词汇索引，将每个索引对应的词汇转换为之前定义好的output_dim维的向量表示。
# 然后将经过嵌入层处理后得到的结果（即所有词汇对应的向量表示组成的张量）打印出来，以便查看词汇经过嵌入层转换后的向量形式以及维度等信息。

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- An embedding layer is essentially a look-up operation:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **You may be interested in the bonus content comparing embedding layers with regular linear layers: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 Encoding word positions

- Embedding layer convert IDs into identical vector representations regardless of where they are located in the input sequence:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- Positional embeddings are combined with the token embedding vector to form the input embeddings for a large language model:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- The BytePair encoder has a vocabulary size of 50,257:
- Suppose we want to encode the input tokens into a 256-dimensional vector representation:

In [47]:
vocab_size = 50257
# 定义一个变量vocab_size，并赋值为50257，表示词汇表的大小为50257，即总共包含50257个不同的词汇（或其他可编码的文本单元等）。
output_dim = 256
# 定义一个变量output_dim，并赋值为256，表示输出的维度为256，这个输出维度通常与后续要进行的某种操作（比如经过神经网络层后的输出维度等）相关。
# 在这里，它表示每个词汇经过嵌入层处理后将被映射到一个256维的向量空间中，通过将词汇转换为高维向量，可以更好地捕捉词汇的语义等信息，以便在后续的神经网络处理中进行更有效的运算和分析等操作。
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# 创建一个嵌入层（Embedding Layer）对象token_embedding_layer。
# 这里使用torch.nn模块中的Embedding类来创建嵌入层。
# 传入的参数vocab_size表示词汇表的大小，即要嵌入的词汇数量；output_dim表示嵌入后的输出维度，即每个词汇经过嵌入层处理后将被映射到一个output_dim维的向量空间中。
# 嵌入层的作用通常是将离散的词汇（或其他可编码的文本单元等）转换为连续的向量表示，以便在后续的神经网络处理中更好地进行运算和分析等操作。
# 在这个例子中，它会将词汇表中的50257个词汇（或文本单元）分别转换为256维的向量，为后续的自然语言处理相关任务（如文本生成、语义理解等）做好数据准备。

- If we sample data from the dataloader, we embed the tokens in each batch into a 256-dimensional vector
- If we have a batch size of 8 with 4 tokens each, this results in a 8 x 4 x 256 tensor:

In [48]:
max_length = 4
# 定义一个变量max_length，并赋值为4，这个变量将用于指定数据样本的最大长度以及滑动窗口的步长。
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
# 调用create_dataloader_v1函数来创建一个数据加载器（dataloader）对象。
# 传入原始文本raw_text作为要处理的数据内容。
# 同时设置了以下参数：
# - batch_size = 8：表示每个批次将包含8个样本，这样每次从数据集中获取数据时会一次性获取8个经过处理的样本数据。
# - max_length = max_length：这里将数据样本的最大长度设置为前面定义的max_length变量的值，即4。意味着数据集中划分出的每个片段长度最多为4个编码单位（例如可能是4个单词经过编码后的形式）。
# - stride = max_length：将滑动窗口的步长也设置为前面定义的max_length变量的值，即4。这意味着在对原始文本进行划分成片段时，相邻片段之间的重叠部分为4个编码单位，由于步长与最大长度相同，可能会导致划分出的片段之间没有重叠或者有特殊的重叠情况，具体取决于原始文本的长度和编码方式等因素。
# - shuffle = False：不打乱数据集中样本的顺序，按照原始文本在经过划分等操作后的顺序来获取数据。
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
# 使用next函数从迭代器data_iter中获取下一个批次的数据。
# 由于前面设置batch_size = 8，所以这里获取到的就是一个包含8个样本数据的批次，并且将这个批次中的输入数据部分赋值给inputs变量，目标数据部分赋值给targets变量。
# 具体来说，inputs和targets的格式和内容取决于create_dataloader_v1函数内部创建数据集时的设置，通常inputs是用于模型输入的编码数据，targets是模型要预测的目标编码数据。

In [49]:
print("Token IDs:\n", inputs)
# 打印出输入数据的信息，先输出提示字符串 "Token IDs:\n"，用于明确输出内容是关于输入数据的编码标识（token IDs）。
# 然后输出变量inputs的值，这里的inputs应该是经过前面一系列处理（如文本编码、数据加载器获取批次数据等操作）得到的输入数据，
print("\nInputs shape:\n", inputs.shape)
# 打印出输入数据的形状信息，先输出提示字符串 "\nInputs shape:\n"，用于明确接下来输出的是输入数据的形状。
# 然后输出变量inputs的形状属性（shape），通过查看形状信息可以了解输入数据在不同维度上的规模情况，比如批次大小、序列长度等维度的具体数值，

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
# 调用之前创建的token_embedding_layer（这是一个torch.nn.Embedding类型的嵌入层对象），并将inputs作为参数传入。
# 嵌入层token_embedding_layer会根据传入的这些词汇索引，将每个索引对应的词汇转换为之前定义好的output_dim维的向量表示。
print(token_embeddings.shape)
# 打印出token_embeddings的形状信息。
# 例如，如果inputs的形状是 [batch_size, sequence_length]（batch_size表示批次大小，sequence_length表示序列长度），

torch.Size([8, 4, 256])


- GPT-2 uses absolute position embeddings, so we just create another embedding layer:

In [51]:
context_length = max_length
# 将之前定义的max_length的值赋给context_length变量。
# 这里的context_length通常用于表示上下文的长度，它与后续要创建的位置嵌入层（pos_embedding_layer）相关，
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# 创建一个位置嵌入层（pos_embedding_layer）对象。
# 使用torch.nn模块中的Embedding类来创建，它与之前创建词汇嵌入层（token_embedding_layer）的方式类似。
# 传入的参数context_length表示要嵌入的位置数量，即根据上下文长度来确定需要生成多少种不同的位置嵌入向量。
# output_dim表示嵌入后的输出维度，即每个位置经过嵌入层处理后将被映射到一个output_dim维的向量空间中。
# 位置嵌入层的作用是为序列中的每个位置赋予一个特定的向量表示，以便模型能够捕捉到序列中不同位置的信息，

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
# 调用之前创建的位置嵌入层对象pos_embedding_layer，并将torch.arange(max_length)作为参数传入。
# torch.arange(max_length)会生成一个从0到max_length - 1的一维张量，这个张量表示了一系列连续的位置索引。
# 位置嵌入层pos_embedding_layer会根据传入的这些位置索引，为每个索引对应的位置生成一个output_dim维的向量表示。
# 然后将经过位置嵌入层处理后得到的所有位置对应的向量表示组成的张量赋值给pos_embeddings变量，以便后续对这些向量进行进一步的处理或分析等操作。
print(pos_embeddings.shape)
# 打印出pos_embeddings的形状信息。
# pos_embeddings是经过位置嵌入层处理后得到的结果，通过查看其形状可以了解到经过位置嵌入处理后的数据在不同维度上的规模情况。
# 由于传入pos_embedding_layer的是一个长度为max_length的一维张量（torch.arange(max_length)），并且位置嵌入层的output_dim为之前定义的某个值，
# 所以pos_embeddings的形状应该为 [max_length, output_dim]

torch.Size([4, 256])


- To create the input embeddings used in an LLM, we simply add the token and the positional embeddings:

In [53]:
input_embeddings = token_embeddings + pos_embeddings
# 将之前得到的token_embeddings（词汇嵌入向量组成的张量，经过token_embedding_layer对输入数据处理得到）
# 和pos_embeddings（位置嵌入向量组成的张量，经过pos_embedding_layer对位置索引处理得到）进行对应元素相加操作。
print(input_embeddings.shape)
# 打印出input_embeddings的形状信息。
# 由于input_embeddings是由token_embeddings和pos_embeddings相加得到的，而token_embeddings的形状通常是由输入数据的批次大小、序列长度和输出维度决定（例如[batch_size, sequence_length, output_dim]），
# pos_embeddings的形状是由上下文长度和输出维度决定，
# 在进行对应元素相加时，要求两者形状在除了最后一个维度（即输出维度）之外的其他维度上要匹配，所以相加后input_embeddings的形状应该和token_embeddings的形状相同。

torch.Size([8, 4, 256])


- In the initial phase of the input processing workflow, the input text is segmented into separate tokens
- Following this segmentation, these tokens are transformed into token IDs based on a predefined vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# Summary and takeaways

See the [./dataloader.ipynb](./dataloader.ipynb) code notebook, which is a concise version of the data loader that we implemented in this chapter and will need for training the GPT model in upcoming chapters.

See [./exercise-solutions.ipynb](./exercise-solutions.ipynb) for the exercise solutions.